In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My\ Drive/PRML-2020/Data\ Contest

/content/drive/My Drive/PRML-2020/Data Contest


In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import pickle as pkl
from geopy.geocoders import Nominatim

# Train set preparation

In [4]:
train = pd.read_csv('./dataset/train.csv')
tour_info = pd.read_csv('./dataset/tours.csv')
tour_info.rename({'biker_id':'host_id', 'latitude':'tour_latitude', 'longitude':'tour_longitude'}, axis=1, inplace=True)
biker_info = pd.read_csv('./dataset/bikers.csv')

train_complete = train.merge(right=biker_info, how='left', on='biker_id')
train_complete = train_complete.merge(right=tour_info, how='left', on='tour_id')
display(train_complete)

,biker_id,tour_id,invited,timestamp,like,dislike,language_id,location_id,bornIn,gender,member_since,area,time_zone,host_id,tour_date,city,state,pincode,country,tour_latitude,tour_longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,...,w62,w63,w64,w65,w66,w67,w68,w69,w70,w71,w72,w73,w74,w75,w76,w77,w78,w79,w80,w81,w82,w83,w84,w85,w86,w87,w88,w89,w90,w91,w92,w93,w94,w95,w96,w97,w98,w99,w100,w_other
0,DA44012,QY18771225,0,02-10-2012 15:53:05,0,0,id,ID,1990,male,02-10-2012,Binjai,480.0,EB06419938,03-10-2012,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
1,DA44012,QU02284248,0,02-10-2012 15:53:05,0,0,id,ID,1990,male,02-10-2012,Binjai,480.0,CA16654644,03-10-2012,Yogyakarta,NaN,NaN,Indonesia,-7.767,110.363,2,0,0,0,2,3,0,0,1,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24
2,DA44012,RU29072432,0,02-10-2012 15:53:05,1,0,id,ID,1990,male,02-10-2012,Binjai,480.0,DG39934255,26-10-2012,Medan,NaN,NaN,Indonesia,3.567,98.650,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,37
3,DA44012,SP72478280,0,02-10-2012 15:53:05,0,0,id,ID,1990,male,02-10-2012,Binjai,480.0,JH461525,06-10-2012,NaN,NaN,NaN,NaN,34.017,71.583,1,2,1,1,0,0,2,0,0,1,1,1,2,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0,56
4,DA44012,QS90707377,0,02-10-2012 15:53:05,0,0,id,ID,1990,male,02-10-2012,Binjai,480.0,DG39934255,06-10-2012,Medan,NaN,NaN,Indonesia,3.607,98.653,2,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13861,EC93103086,RW50873665,0,08-12-2012 03:59:43,0,0,id,ID,1993,female,28-10-2012,Yogyakarta 10,420.0,HB6641327,08-12-2012,Bandung,NaN,NaN,Indonesia,-6.973,107.633,1,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80
13862,EC93103086,TP84655790,0,08-12-2012 03:59:43,0,0,id,ID,1993,female,28-10-2012,Yogyakarta 10,420.0,JC4601231,09-12-2012,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80
13863,EC93103086,UY8708806,0,08-12-2012 03:59:43,0,0,id,ID,1993,female,28-10-2012,Yogyakarta 10,420.0,BD43473057,09-12-2012,Malang,NaN,NaN,Indonesia,-7.973,112.609,3,2,0,1,1,0,0,0,1,0,0,0,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,60
13864,EC93103086,VP4179853,0,08-12-2012 03:59:43,0,0,id,ID,1993,female,28-10-2012,Yogyakarta 10,420.0,BF40022070,12-12-2012,Jember,Jawa Timur,NaN,Indonesia,-8.167,113.700,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24


In [5]:
time_zone = []
for index in tqdm(train_complete.index):
    biker_id = train_complete.iloc[index]['biker_id']
    time = train_complete.iloc[index]['time_zone']
    area = train_complete.iloc[index]['area']
    location_id = train_complete.iloc[index]['location_id']
    if np.isnan(time):
        if area == 'Epsom':
            time_zone.append(60)
        elif 'Yogyakarta' in area:
            time_zone.append(420)
        elif 'Los Angeles' in area:
            time_zone.append(-480)
        elif 'Abuja' in area:
            time_zone.append(60)
        elif 'London' in area:
            time_zone.append(60)
        elif 'Sigli  Aceh' in area:
            time_zone.append(420)
        elif 'San Francisco' in area:
            time_zone.append(-420)
        elif 'Liverpool' in area:
            time_zone.append(240)
        elif location_id == 'FR':
            time_zone.append(60)
        elif location_id == 'US':
            time_zone.append(-240)
    else:
        time_zone.append(time)

In [6]:
tour_day = []
tour_month = []
tour_year = []
biker_bornIn = []
biker_member_since_day = []
biker_member_since_month = []
biker_member_since_year = []
tour_latitude = []
tour_longitude = []
locator = Nominatim(user_agent='myGeocoder')
format = '%d-%m-%Y'

for index in tqdm(train_complete.index):
    date = train_complete.iloc[index]['tour_date']
    cty = train_complete.iloc[index]['city']
    stt = train_complete.iloc[index]['state']
    ctry = train_complete.iloc[index]['country']
    ltt = train_complete.iloc[index]['tour_latitude']
    lgt = train_complete.iloc[index]['tour_longitude']
    mdate = train_complete.iloc[index]['member_since']
    byear = train_complete.iloc[index]['bornIn']

    address = ''
    if isinstance(cty, str):
        address += (cty + ',')
    if isinstance(stt, str):
        address += (stt + ',')
    if isinstance(ctry, str):
        address += ctry

    if np.isnan(ltt) or np.isnan(lgt):
        if address:
            try:
                loc = locator.geocode(address)
                tour_latitude.append(loc.latitude)
                tour_longitude.append(loc.longitude)
            except:
                tour_latitude.append(0)
                tour_longitude.append(0)

        else:
            tour_latitude.append(0)
            tour_longitude.append(0)
    else:
        tour_latitude.append(ltt)
        tour_longitude.append(lgt)

    if byear == 'None':
        biker_bornIn.append(1952)
    else:
        biker_bornIn.append(int(byear))

    dt = datetime.strptime(date, format)
    tour_day.append(dt.day)
    tour_month.append(dt.month)
    tour_year.append(dt.year)

    dt = datetime.strptime(mdate, format)
    biker_member_since_day.append(dt.day)
    biker_member_since_month.append(dt.month)
    biker_member_since_year.append(dt.year)

In [7]:
bikers_network = pd.read_csv('./dataset/bikers_network.csv')
bikers_network.index = bikers_network['biker_id']
bikers_network.drop(['biker_id'], axis=1, inplace=True)
display(bikers_network)

,friends
biker_id,
DB97468391,BD46449342 DI73244116 EC26080662 BC22907620 FE...
DF37982273,BE91560444 DJ5798035 CA36380346 IJ9375619 DF34...
IC3183725,BE84954627 BJ50387873 BG52977611 EB85960823 EC...
BI72223848,ID361640 HC3814682 FF7944478 BH24049724 CF3059...
DE29017717,EC53303705 CB30310957 BI38389374 DJ28735761 HB...
...,...
CD21558820,GB9730283 DB56233255 BH27834076 CD22451089 DI5...
EA10747551,BF52872602 DA85585165 BF57664371 CH43735210 CB...
CI0076614,EA83058327 IB2658711 BC22305296 JF3357686 DC42...


In [8]:
tour_convoy = pd.read_csv('./dataset/tour_convoy.csv')
tour_convoy.index = tour_convoy['tour_id']
tour_convoy.drop(['tour_id'], axis=1, inplace=True)
display(tour_convoy)

,going,maybe,invited,not_going
tour_id,,,,
QQ59822043,BJ75964455 CF2302513 EC26086795 DI05886383 BE2...,CH33420590 FB7546982 BD50834692 FD2087573 FI31...,BH23091036 DH95873583 EB09144917 DF60622906 DB...,DF75574655 BA77296663
VX6467261,CD94228942 CG86116898 BA56558062 DH92942231 EB...,BE98184352 GE5689144 DH70076778 DD1335845 EC39...,BH88073374 HD3302094 BI30571649 GH6508092 HA81...,NaN
QQ86208412,NaN,DD20380166 DI10793697,BD79121209 EE0668682,BH28988561 CJ50720854
RV21578336,NaN,NaN,NaN,NaN
XU5842686,CE06118796 DF50897984 CJ4255260 BB25817077 BA9...,CG71721559 BH61448345 CD56975806 CG66669465 BA...,BF18670705 II0919237 CD26414227 CG73818347 DD2...,DF00235232
...,...,...,...,...
QQ0980591,NaN,JB9719690 FG3897585 BG60054065,EC40518642 BG00401182 BA69815686 FH5688453 BG8...,NaN
UX2973300,CH64820512 FH535535 DI52680355 BF52352413 BJ22...,BH50292668,EC68940236 IE3418792 CE32405737 EJ6887289 CD17...,DI95101160
QR94481466,EB97808605 CB09654543 BE11365919 DD12623074 CI...,BI15047818 DA7909158 DJ28453636 DE25034044 EI9...,DG52885869 BE02077635 EA94813681 BE81916408 JE...,DF21355909


In [9]:
num_friends_going = []
num_friends_not_going = []
num_friends_maybe = []
num_friends_invited = []

for index in tqdm(train_complete.index):
    tour_id = train_complete.iloc[index]['tour_id']
    biker_id = train_complete.iloc[index]['biker_id']

    try:
        friends = bikers_network.loc[biker_id]['friends'].split()
    except:
        friends = []
    try:
        going = tour_convoy.loc[tour_id]['going'].split()
    except:
        going = []
    try:
        not_going = tour_convoy.loc[tour_id]['not_going'].split()
    except:
        not_going = []
    try:
        maybe = tour_convoy.loc[tour_id]['maybe'].split()
    except:
        maybe = []
    try:
        invited = tour_convoy.loc[tour_id]['invited'].split()
    except:
        invited = []

    num_going, num_not_going, num_maybe, num_invited = 0, 0, 0, 0
    for friend in friends:
        if friend in going:
            num_going += 1
        elif friend in not_going:
            num_not_going += 1
        elif friend in maybe:
            num_maybe += 1
        if friend in invited:
            num_invited += 1
        
    num_friends_going.append(num_going)
    num_friends_not_going.append(num_not_going)
    num_friends_maybe.append(num_maybe)
    num_friends_invited.append(num_invited)

In [10]:
train_complete.drop(['timestamp', 'language_id', 'location_id', 'member_since', 
                     'tour_date', 'area', 'city', 'state', 'country', 'pincode'], axis=1, inplace=True)
train_complete['bornIn'] = biker_bornIn
train_complete['tour_day'] = tour_day
train_complete['tour_month'] = tour_month
train_complete['tour_year'] = tour_year
train_complete['tour_latitude'] = tour_latitude
train_complete['tour_longitude'] = tour_longitude
train_complete['member_since_day'] = biker_member_since_day
train_complete['member_since_month'] = biker_member_since_month
train_complete['member_since_year'] = biker_member_since_year
train_complete['time_zone'] = time_zone
train_complete['num_friends_going'] = num_friends_going
train_complete['num_friends_not_going'] = num_friends_not_going
train_complete['num_friends_maybe'] = num_friends_maybe
train_complete['num_friends_invited'] = num_friends_invited
train_complete = pd.get_dummies(train_complete, columns=['gender'])
display(train_complete)

,biker_id,tour_id,invited,like,dislike,bornIn,time_zone,host_id,tour_latitude,tour_longitude,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12,w13,w14,w15,w16,w17,w18,w19,w20,w21,w22,w23,w24,w25,w26,w27,w28,w29,w30,...,w74,w75,w76,w77,w78,w79,w80,w81,w82,w83,w84,w85,w86,w87,w88,w89,w90,w91,w92,w93,w94,w95,w96,w97,w98,w99,w100,w_other,tour_day,tour_month,tour_year,member_since_day,member_since_month,member_since_year,num_friends_going,num_friends_not_going,num_friends_maybe,num_friends_invited,gender_female,gender_male
0,DA44012,QY18771225,0,0,0,1990,480.0,EB06419938,0.000,0.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,10,2012,2,10,2012,0,3,0,3,0,1
1,DA44012,QU02284248,0,0,0,1990,480.0,CA16654644,-7.767,110.363,2,0,0,0,2,3,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,3,10,2012,2,10,2012,0,0,0,1,0,1
2,DA44012,RU29072432,0,1,0,1990,480.0,DG39934255,3.567,98.650,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,37,26,10,2012,2,10,2012,0,0,0,2,0,1
3,DA44012,SP72478280,0,0,0,1990,480.0,JH461525,34.017,71.583,1,2,1,1,0,0,2,0,0,1,1,1,2,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0,56,6,10,2012,2,10,2012,1,0,0,0,0,1
4,DA44012,QS90707377,0,0,0,1990,480.0,DG39934255,3.607,98.653,2,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,79,6,10,2012,2,10,2012,0,0,0,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13861,EC93103086,RW50873665,0,0,0,1993,420.0,HB6641327,-6.973,107.633,1,0,0,0,0,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,8,12,2012,28,10,2012,0,0,1,0,1,0
13862,EC93103086,TP84655790,0,0,0,1993,420.0,JC4601231,0.000,0.000,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,80,9,12,2012,28,10,2012,1,0,0,1,1,0
13863,EC93103086,UY8708806,0,0,0,1993,420.0,BD43473057,-7.973,112.609,3,2,0,1,1,0,0,0,1,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,60,9,12,2012,28,10,2012,1,1,1,3,1,0
13864,EC93103086,VP4179853,0,0,0,1993,420.0,BF40022070,-8.167,113.700,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,12,12,2012,28,10,2012,0,1,1,0,1,0


In [11]:
train_complete.to_csv('./Required data/train_complete.csv')